In [6]:
!pwd

/media/jack/images/flask_conversations


In [1]:
import json
import logging
import os

def split_and_save_conversations(conversations_file, output_folder):
    try:
        with open(conversations_file, 'r', encoding='utf-8') as file:
            data = json.load(file)
            
            for conversation in data:
                title = conversation.get('title', 'Unknown_Title')
                title_with_underscores = title.replace(' ', '_')
                chapter_filename = f"{title_with_underscores}.json"
                chapter_filepath = os.path.join(output_folder, chapter_filename)
                
                logging.info(f"Saving data for conversation '{title}' to {chapter_filepath}")
                
                with open(chapter_filepath, 'w', encoding='utf-8') as chapter_file:
                    json.dump([conversation], chapter_file, indent=2)

    except FileNotFoundError:
        logging.error(f"File not found: {conversations_file}")
    except json.JSONDecodeError:
        logging.error(f"Error decoding JSON in file: {conversations_file}")
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")

# Example usage
conversations_file_path = '/media/jack/images/flask_conversations/CHATDPT/conversations.json'
output_folder = '/media/jack/images/flask_conversations/CHATDPT/chapters_indexed'

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Configure logging
logging.basicConfig(level=logging.INFO)

# Call the split and save function
split_and_save_conversations(conversations_file_path, output_folder)


INFO:root:Saving data for conversation 'Good Conversations Guide' to /media/jack/images/flask_conversations/CHATDPT/chapters_indexed/Good_Conversations_Guide.json
INFO:root:Saving data for conversation 'Arcanians: Diverse, Adaptive Ecosystem' to /media/jack/images/flask_conversations/CHATDPT/chapters_indexed/Arcanians:_Diverse,_Adaptive_Ecosystem.json
INFO:root:Saving data for conversation 'Free Crypto Wallets Download' to /media/jack/images/flask_conversations/CHATDPT/chapters_indexed/Free_Crypto_Wallets_Download.json


In [2]:
%%writefile json2html.py
#!/home/jack/miniconda3/envs/cloned_base/bin/python
import json
from sys import argv
# Load the JSON data from the uploaded file
DIR="/media/jack/images/flask_conversations/CHATDPT/chapters_indexed/"
filename=argv[1]
Filename = DIR+filename
print(Filename)
with open(Filename, 'r') as file:
    json_data = json.load(file)

# Initialize the result string
result_str = ""

# Define a function to get conversation messages similar to the JavaScript logic
def get_conversation_messages(conversation):
    messages = []
    current_node = conversation.get('current_node')
    while current_node:
        node = conversation['mapping'][current_node]
        message = node.get('message')
        if (message and message.get('content') and message['content'].get('content_type') == 'text' and
            len(message['content'].get('parts', [])) > 0 and len(message['content']['parts'][0]) > 0 and
                (message['author']['role'] != 'system' or message.get('metadata', {}).get('is_user_system_message'))):
            author = message['author']['role']
            if author == 'assistant':
                author = 'ChatGPT'
            elif author == 'system' and message['metadata'].get('is_user_system_message'):
                author = 'Custom user info'
            messages.append({'author': author, 'text': message['content']['parts'][0]})
        current_node = node.get('parent')
    return messages[::-1]  # Reverse the list to maintain chronological order

# Iterate over each conversation in the JSON data and process it
for conversation in json_data:
    # Get the conversation title and messages
    title = conversation.get('title', '')
    messages = get_conversation_messages(conversation)

    # Append the title and messages to the result string
    result_str += title + '\n'
    for message in messages:
        result_str += message['author'] + '\n' + message['text'] + '\n'
    result_str += '\n'  # Add a newline between conversations

# Return the processed result string
print(result_str.strip())
HTMLfile=DIR+filename[:-4]+"html"
print("HTMLfile: ",HTMLfile)
Input = open(HTMLfile,"w")
result_str = result_str.replace("/n","XXXXXXX\n")
result_str = result_str.replace("<","&lt;")
result_str = result_str.replace(">","&gt;")
for line in result_str.split("XXXXXXX"):
    line = line.replace("\n","<br />\n")
    Input.write(line)
Input.close()

Overwriting json2html.py


In [3]:
!chmod +x json2html.py


In [3]:
#!/home/jack/miniconda3/envs/cloned_base/bin/python
import json
import os
import glob
import logging

# Set up logging configuration
logging.basicConfig(filename='conversion_log.txt', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define the directory path
LOCAL=os.getcwd()
DIR = LOCAL+"/CHATDPT/chapters_indexed/"
print(DIR)
# List all JSON files in the directory
jsonfiles = glob.glob(os.path.join(DIR, '*.json'))
print(jsonfiles)
if not jsonfiles:
    logging.warning("No JSON files found in the specified directory.")
    exit()

# Iterate over each JSON file in the directory
for json_file in jsonfiles:
    # Load the JSON data from the current file
    with open(json_file, 'r') as file:
        json_data = json.load(file)

    # Define the HTML file path based on the current JSON file
    HTMLfile = os.path.join(DIR, os.path.basename(json_file)[:-5] + ".html")

    print("HTMLfile: ", HTMLfile)

    # Initialize the result string for current JSON file
    result_str = ""

#!/home/jack/miniconda3/envs/cloned_base/bin/python
import json
import os
import glob
import logging

# Set up logging configuration
logging.basicConfig(filename='conversion_log.txt', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define the directory path
DIR = "CHATDPT/chapters_indexed/"

# List all JSON files in the directory
jsonfiles = glob.glob(os.path.join(DIR, '*.json'))
print(jsonfiles)
if not jsonfiles:
    logging.warning("No JSON files found in the specified directory.")
    exit()

# Iterate over each JSON file in the directory
for json_file in jsonfiles:
    # Load the JSON data from the current file
    with open(json_file, 'r') as file:
        json_data = json.load(file)

    # Define the HTML file path based on the current JSON file
    HTMLfile = os.path.join(DIR, os.path.basename(json_file)[:-5] + ".html")

    print("HTMLfile: ", HTMLfile)

    # Initialize the result string for current JSON file
    result_str = ""

    def get_conversation_messages(conversation):
        messages = []
        current_node = conversation.get('current_node')
        while current_node:
            node = conversation['mapping'].get(current_node)
            if node:
                message = node.get('message')
                if (message and message.get('content') and message['content'].get('content_type') == 'text' and
                    len(message['content'].get('parts', [])) > 0 and len(message['content']['parts'][0]) > 0 and
                        (message['author']['role'] != 'system' or message.get('metadata', {}).get('is_user_system_message'))):
                    author = message['author']['role']
                    if author == 'assistant':
                        author = 'ChatGPT'
                    elif author == 'system' and message['metadata'].get('is_user_system_message'):
                        author = 'Custom user info'
                    messages.append({'author': author, 'text': message['content']['parts'][0]})
                current_node = node.get('parent')
            else:
                logging.warning(f"Missing node for current_node: {current_node} in conversation.")
                break  # Break the loop if the node is missing

        return messages[::-1] if messages else []  # Return messages or an empty list if no messages found

    # Process each conversation in the JSON data and append to result_str
    for conversation in json_data:
        # Get the conversation title and messages
        title = conversation.get('title', '')
        messages = get_conversation_messages(conversation)

        # Append the title and messages to the result string
        result_str += title + '\n'
        for message in messages:
            result_str += message['author'] + '\n' + message['text'] + '\n'
        result_str += '\n'  # Add a newline between conversations

    # Write the HTML content to the file
    with open(HTMLfile, "w") as Input:
        result_str = result_str.replace("\n", "<br />\n")
        result_str = result_str.replace("<", "&lt;")
        result_str = result_str.replace(">", "&gt;")
        Input.write(result_str)

    print(f"HTML conversion completed for {json_file}.")

# Logging for the entire process completion
logging.info("Conversion process completed for all JSON files.")

logging.info("Conversion process completed for all JSON files.")


INFO:root:Conversion process completed for all JSON files.
INFO:root:Conversion process completed for all JSON files.


/media/jack/images/conversations.json/CHATDPT/chapters_indexed/
['/media/jack/images/conversations.json/CHATDPT/chapters_indexed/Good_Conversations_Guide.json', '/media/jack/images/conversations.json/CHATDPT/chapters_indexed/Arcanians:_Diverse,_Adaptive_Ecosystem.json', '/media/jack/images/conversations.json/CHATDPT/chapters_indexed/Free_Crypto_Wallets_Download.json']
HTMLfile:  /media/jack/images/conversations.json/CHATDPT/chapters_indexed/Good_Conversations_Guide.html
HTMLfile:  /media/jack/images/conversations.json/CHATDPT/chapters_indexed/Arcanians:_Diverse,_Adaptive_Ecosystem.html
HTMLfile:  /media/jack/images/conversations.json/CHATDPT/chapters_indexed/Free_Crypto_Wallets_Download.html
['CHATDPT/chapters_indexed/Good_Conversations_Guide.json', 'CHATDPT/chapters_indexed/Arcanians:_Diverse,_Adaptive_Ecosystem.json', 'CHATDPT/chapters_indexed/Free_Crypto_Wallets_Download.json']
HTMLfile:  CHATDPT/chapters_indexed/Good_Conversations_Guide.html
HTML conversion completed for CHATDPT/ch

# Create Sqlite3  populated with data from HTML files

In [5]:
import sqlite3
import os
import hashlib

# Connect to SQLite database (creates a new database if it doesn't exist)
db_path = 'chat_database.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Create a table to store file information
cursor.execute('''
    CREATE TABLE IF NOT EXISTS files (
        id INTEGER PRIMARY KEY,
        filename TEXT NOT NULL,
        content BLOB NOT NULL,
        text_content TEXT NOT NULL,
        hash_value TEXT NOT NULL,
        format TEXT NOT NULL
    )
''')

# Commit changes and close the connection
conn.commit()
conn.close()

# Function to calculate SHA-256 hash of a file
def calculate_hash(file_path):
    sha256 = hashlib.sha256()
    with open(file_path, 'rb') as file:
        while chunk := file.read(8192):  # Read in 8KB chunks
            sha256.update(chunk)
    return sha256.hexdigest()

# Function to insert a file into the database
def insert_file(filename, content, text_content, hash_value, file_format):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute('INSERT INTO files (filename, content, text_content, hash_value, format) VALUES (?, ?, ?, ?, ?)',
                   (filename, content, text_content, hash_value, file_format))
    conn.commit()
    conn.close()

# Function to insert HTML files recursively
def insert_html_files(directory):
    for root, _, files in os.walk(directory):
        for file_name in files:
            if file_name.endswith('.html'):
                file_path = os.path.join(root, file_name)
                with open(file_path, 'rb') as file:
                    file_content = file.read()
                text_content = file_content.decode('utf-8', errors='ignore')  # Convert bytes to string
                hash_value = calculate_hash(file_path)
                insert_file(file_name, file_content, text_content, hash_value, 'html')
                print(f"Inserted: {file_name}")

# Example: Insert HTML files recursively from the specified directory
insert_html_files('CHATDPT/')

print('Insertion process completed.')


Inserted: chat.html
Inserted: Arcanians:_Diverse,_Adaptive_Ecosystem.html
Inserted: Good_Conversations_Guide.html
Inserted: Free_Crypto_Wallets_Download.html
Insertion process completed.


# Database search _ Working Good

In [6]:
import sqlite3
import uuid

# Connect to SQLite database
db_path = 'chat_database.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

def retrieve_file_content(filename):
    cursor.execute('SELECT content FROM files WHERE filename = ?', (filename,))
    result = cursor.fetchone()
    return result[0] if result else None

def search_and_print_fourth_file(search_terms):
    Data = ""

    # Prepare the SQL query for searching files based on the given terms
    query = '''
        SELECT filename
        FROM files
        WHERE {}
    '''.format(' AND '.join(['text_content LIKE ?' for _ in search_terms]))

    # Add % around search terms for a partial match with spaces
    search_terms = ['% {} %'.format(term) for term in search_terms]

    # Execute the query and retrieve matching files
    cursor.execute(query, search_terms)
    matching_files = cursor.fetchall()

    # Check if there are at least 2 matching files
    if matching_files and len(matching_files) >= 2:
        fourth_file = matching_files[1][0]  # Get the filename of the second matching file
        print(fourth_file)

        # Retrieve the content of the matching file
        content = retrieve_file_content(fourth_file)
        
        if content:
            # Decode the content and append it to the Data variable
            Data = Data + f'{content.decode("utf-8", errors="ignore")}'
            print(Data)
            return Data
        else:
            print(f'Error: Content not found for {fourth_file}')
    else:
        print('Error: No matching files found or less than two matching files.')

# Example: Search for files containing 'flask' and '5200'
search_terms = ['Cephalux','Morpholux']
DATA = search_and_print_fourth_file(search_terms)

# Close the connection to the database
conn.close()

# If data is found, create a unique filename and write the content to an HTML file
if len(DATA) > 2:
    uid = str(uuid.uuid4())  # Generate a unique ID using uuid
    FileName = "_".join(search_terms) + "_" + uid + ".html"
    print(FileName)

    # Open the file for writing
    with open(FileName, "w") as IN:
        # Split the data into lines and write each line to the file with "<br />" appended
        ndata = DATA.split("<br />\n")
        for line in ndata:
            print(line)
            IN.write(line + "<br />\n")


Arcanians:_Diverse,_Adaptive_Ecosystem.html
Arcanians: Diverse, Adaptive Ecosystem&lt;br /&gt;
user&lt;br /&gt;
In the vast and sparsely populated world of Arcanis, the dominant life forms are collectively known as Arcanians. These beings are categorized into three distinct species, each adapted to a specific environment on the planet.&lt;br /&gt;
&lt;br /&gt;
    Terrianis: Desert Dwellers&lt;br /&gt;
        Terrianis inhabit the majority of Arcanis, enduring the harsh conditions of the expansive deserts that dominate the land. Nomadic in nature, they spend their days navigating the challenging terrain, struggling for survival.&lt;br /&gt;
&lt;br /&gt;
    Cephalux: Ocean Dwellers&lt;br /&gt;
        Cephalux, akin to Earth's cephalopods, reside in the depths of Arcanis' oceans. While their aquatic lifestyle limits their mechanical and manufacturing capabilities, Cephalux exhibit remarkable adaptability within their underwater realm.&lt;br /&gt;
&lt;br /&gt;
    Morpholux: Land and S

## Explanation and Comments:

### Database Connection:
- Establishes a connection to the SQLite database.

### Function `retrieve_file_content`:
- Retrieves the content of a file from the database based on the filename.

### Function `search_and_print_fourth_file`:
- Searches for files based on given search terms using SQL LIKE.
- Retrieves the second matching file (index 1) and its content.
- Appends the decoded content to the `Data` variable.

### Example Search:
- Searches for files containing 'flask' and '5200'.

### Close Connection:
- Closes the connection to the database.

### Create Unique Filename and Write HTML File:
- If data is found, generates a unique filename with search terms and a UUID.
- Opens the file for writing and writes each line of the content with "<br />" appended.

This script is designed to search for files based on certain terms, retrieve the content of the second matching file, and then create an HTML file with a unique filename and the retrieved content. Feel free to ask if you have any specific questions or if there's anything else you'd like to understand!
